---
### Подготовка

Импорт библиотек

In [36]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from IPython.display import display
plt.rc('font', family='Verdana')
from sklearn.decomposition import PCA, KernelPCA
import mlxtend as ml
from mlxtend.frequent_patterns import apriori
import itertools
from typing import Dict, List
from mlxtend.preprocessing import TransactionEncoder
encoder = TransactionEncoder()


<br />
Подготовка данных из задания

Дан набор данных:

| tid              | itemset |
| ---------------- | ------- |
| *t<sub>1</sub>*  | *ABC*    |
| *t<sub>2</sub>*  | *AD*    |
| *t<sub>3</sub>*  | *AE*    |
| *t<sub>4</sub>*  | *AD*    |
| *t<sub>5</sub>*  | *ABC*    |
| *t<sub>6</sub>*  | *AE*    |
| *t<sub>7</sub>*  | *BCD*    |
| *t<sub>8</sub>*  | *ADE*    |

Минимальный уровень поддержки = **1**

In [37]:
tid = ['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8']
itemset = ['ABC', 'AD', 'AE', 'AD', 'ABC', 'AE', 'BCD', 'ADE']
min_support = 0.125

### Задание №1 

**1.1**. Найдите частые наборы. Для поиска воспользуйтесь собственной реализацией **Apriori** из работы №3.<br /><br />

In [38]:
def apriori(df: pd.DataFrame, min_support: float) -> Dict[int, List[List[str]]]:
    transaction_matrix = df.values
    transaction_count = transaction_matrix.shape[0]
    support_counts = np.array(np.sum(transaction_matrix, axis=0) / transaction_count)
    item_id_to_name = {i: item for i, item in enumerate(df.columns)}
    item_ids = np.arange(transaction_matrix.shape[1])
    item_set_supports = {1: support_counts[support_counts >= min_support]}
    frequent_item_sets = {1: item_ids[support_counts >= min_support].reshape(-1, 1)}

    k = 2
    while True:
        new_item_sets = []
        leaves = set(frequent_item_sets[k-1].flatten())
        for item_set in frequent_item_sets[k-1]:
            max_item_id = max(item_set)
            new_leaves = [x for x in leaves if x > max_item_id]
            last_tuple = tuple(item_set)
            for new_leaf in new_leaves:
                new_item_sets.append(last_tuple + (new_leaf,))
        
        new_item_sets = np.array(new_item_sets)
        if (new_item_sets.size == 0):
            break
        
        bool_mask = np.array(np.all(transaction_matrix[:, new_item_sets], axis=2))
        support_counts = np.array(np.sum(bool_mask, axis=0) / transaction_count)
        mask = (support_counts >= min_support)
        if mask.any():
            item_set_supports[k] = np.array(support_counts[mask])
            frequent_item_sets[k] = np.array(new_item_sets[mask])
        else:
            break
        
        k += 1
        
    result = {}
    for k in frequent_item_sets:
        for item_set, support in zip(frequent_item_sets[k], item_set_supports[k]):
            result[tuple(item_id_to_name[i] for i in item_set)] = support
    return result

encoder.fit(itemset)
encodedData = encoder.transform(itemset)
data = pd.DataFrame(encodedData, columns = encoder.columns_, index = tid)
apriori(data, min_support)

{('A',): 0.875,
 ('B',): 0.375,
 ('C',): 0.375,
 ('D',): 0.5,
 ('E',): 0.375,
 ('A', 'B'): 0.25,
 ('A', 'C'): 0.25,
 ('A', 'D'): 0.375,
 ('A', 'E'): 0.375,
 ('B', 'C'): 0.375,
 ('B', 'D'): 0.125,
 ('C', 'D'): 0.125,
 ('D', 'E'): 0.125,
 ('A', 'B', 'C'): 0.25,
 ('A', 'D', 'E'): 0.125,
 ('B', 'C', 'D'): 0.125}

**1.2**. Найдите все минимальные генераторы. Оформите этот пункт в виде функции.<br />

In [39]:
Data = ['ABC', 'AD', 'AE', 'AD', 'ABC', 'AE', 'BCD', 'ADE']
Alphabet = [];
min_sup = 1;
min_word=[]

def CheckAlphabet(element):
    for i in range(len(Alphabet)):
        if(Alphabet[i]==element):
            return False
    return True

def CreateAlphabet(data):
    for i in range(len(data)):
        for j in range(len(data[i])):
            if(len(Alphabet)==0 or CheckAlphabet(data[i][j])==True):
                Alphabet.append(data[i][j])

def find_el(str1,str2)->int:
    k = 0;
    for i in range(len(str2)):
        for j in range(len(str1)):
            if(str2[i] == str1[j]):
                k=k+1;
                break;
    return k;

def analis(data,str1)->int:
    k = 0;
    for i in range(len(data)):
        if(find_el(str1,data[i])/len(str1)==1.0):
            k = k + 1;
    return k;

def a(min_,s,data):
    if(len(min_)==0):
        return
    for i in range(len(min_)):
        s = s + min_[i];
        an = analis(data,s)
        if(an>=min_sup):
            min_word.append(s)
            a(min_[i+1:len(min_)],s,data)
        s = s[0:len(s)-1]
    return


def min_len(data):
    min = 1000;
    for i in range(len(data)):
        if(len(data[i])<min):
            min = len(data[i]);
    return min

def delete(data,l):
    d = [];
    for i in range(len(data)):
        if(l==len(data[i])):
            d.append(data[i])
    return d

def z_class(data):
    z_c=[]
    j = 0;
    while(len(data)!=0 ):
        str1 = data[len(data)-1];
        z_c.append([str1])
        
        for i in range(len(data)-1):
            if(find_el(str1,data[i])==len(data[i]) and data[i]!='0'):
                z_c[j].append(data[i])
                data[i]='0'
        j = j + 1
        data.pop()
    new_z_c = []
    for i in range(len(z_c)):
        if(z_c[i]!=['0']):
            new_z_c.append(z_c[i])
    for i in range(len(new_z_c)):
        m_l = min_len(new_z_c[i])
        new_z_c[i] = delete(new_z_c[i],m_l)
        print( str(analis(Data,new_z_c[i][0])) + " : " + str(new_z_c[i]))
    return new_z_c


def min_gen(Data,min_word):
    d = dict()
    for i in range(len(min_word)):
        perm = analis(Data,min_word[i])
        if(d.get(perm)==None):
            d.update({perm:[min_word[i]]})
        else:
            d[analis(Data,min_word[i])].append(min_word[i])
    keys = list(d.keys())
    for i in range(len(keys)):
        d[keys[i]] = sorted(d[keys[i]],key=len)
    return d
CreateAlphabet(Data)
a(Alphabet,'',Data)
print('Минимальные генераторы')
dictionary = min_gen(Data,min_word)
for key, value in dictionary.items():
        dictionary[key] = z_class(dictionary[key])

Минимальные генераторы
7 : ['A']
2 : ['AB', 'AC']
3 : ['B', 'C']
3 : ['E']
3 : ['AD']
1 : ['BD', 'CD']
1 : ['DE']
4 : ['D']


**1.3**. К исходным данным добавьте набор ***AE*** и повторно найдите все минимальные генераторы. Значение минимального уровня поддержки оставьте без изменений.<br />

In [40]:
Data.append('AE')
CreateAlphabet(Data)
min_sup = 1;
min_word=[]
a(Alphabet,'',Data)
dictionary = min_gen(Data,min_word)
print('Минимальные генераторы')
for key, value in dictionary.items():
        dictionary[key] = z_class(dictionary[key])

Минимальные генераторы
8 : ['A']
2 : ['AB', 'AC']
3 : ['B', 'C']
3 : ['AD']
1 : ['BD', 'CD']
1 : ['DE']
4 : ['E']
4 : ['D']


### Задание №2

**2.1**. Используя решетку наборов и их частот, восстановите изначальный набор данных **T**. Отсортируйте **T** по ключу ```(len(Ti), Ti)``` в порядке возрастания<br /><br />

In [41]:
data = ['ABCDE', 'ABCD', 'ABDE', 'ACDE', 'BCDE', 'ACD', 'BCD', 'AC', 'BD', 'CE']
data.sort(key=lambda x:len(x))
T = pd.DataFrame({'items': data})
T

,items
0,AC
1,BD
2,CE
3,ACD
4,BCD
5,ABCD
6,ABDE
7,ACDE
8,BCDE
9,ABCDE



**2.2**. Для каждого набора *X<sub>i</sub>* вычислите множество **M<sub>i</sub>** индексов тидов *t<sub>j</sub>* ∈ **T**, в которых содержится этот набор: *M<sub>i</sub> = {j | X<sub>i</sub> ⊆ t<sub>j</sub>}*. Получите множество **P** *= {(X<sub>i</sub>, M<sub>i</sub>)}*<br /><br />

In [42]:
def get_all_elem(df):
    x = [i for i in df['items']]

    all_elements = []
    for i in df['items']:
        all_elements += list(i)
    all_elements = set(all_elements)
    return sorted(all_elements)
def generate_elem2(elements):
    letters = []
    for i in elements:
        i_list = list(i)
        for j in i_list:
            if j not in letters:
                letters.append(j)
    letters.sort()

    t = []
    for i in elements:
        last_letter = i[-1]
        for j in letters:
            if last_letter < j:
                current_elem = i + j

                t.append(current_elem)

    return sorted(t)
def correct_elem(df, letters):
    result = [[], []]
    for i in letters:
        a = check_elem(df, i)
        result[0].append(a[0])
        result[1].append(a[1])

    if (len(result[0]) == 0):
        return None
    else:
        return result
    
def check_elem(df, elem): 
    count = 0
    indexx = []
    flag = 0
    elem_list = list(elem)
    for index, row in df.iterrows():

       for count_i, i in enumerate(elem_list): 
           if not row[i]:
                flag = 1
                break
       if flag == 0:
           indexx.append(index)
       else:
           flag = 0
    return elem, indexx

def preprocessing(df):
    letters = get_all_elem(df)
    all_elems = []

    for i in df['items']:
        elems = list(i)
        all_elems.append(elems)

    df = pd.DataFrame(columns=letters)
    mask = [False] * len(letters)

    for i, value_i in enumerate(all_elems):
        for j, value_j in enumerate(letters):
            if value_i.count(value_j):
                mask[j] = True
            else:
                mask[j] = False
        df.loc[len(df)] = mask

    return df

In [43]:
def get_index(T):
    res = [[], []]
    x = get_all_elem(T)
    for i in range(1, 6):
        for j in generate_elem2(x):
            if(len(j) == i):
                x.append(j)
    T = preprocessing(T)
    ans = correct_elem(T, x)
    res[0] += ans[0]
    res[1] += ans[1]
    return dict(zip(res[0], res[1]))
index_set = get_index(T)
index_set

{'A': [0, 3, 5, 6, 7, 9],
 'B': [1, 4, 5, 6, 8, 9],
 'C': [0, 2, 3, 4, 5, 7, 8, 9],
 'D': [1, 3, 4, 5, 6, 7, 8, 9],
 'E': [2, 6, 7, 8, 9],
 'AB': [5, 6, 9],
 'AC': [0, 3, 5, 7, 9],
 'AD': [3, 5, 6, 7, 9],
 'AE': [6, 7, 9],
 'BC': [4, 5, 8, 9],
 'BD': [1, 4, 5, 6, 8, 9],
 'BE': [6, 8, 9],
 'CD': [3, 4, 5, 7, 8, 9],
 'CE': [2, 7, 8, 9],
 'DE': [6, 7, 8, 9],
 'ABC': [5, 9],
 'ABD': [5, 6, 9],
 'ABE': [6, 9],
 'ACD': [3, 5, 7, 9],
 'ACE': [7, 9],
 'ADE': [6, 7, 9],
 'BCD': [4, 5, 8, 9],
 'BCE': [8, 9],
 'BDE': [6, 8, 9],
 'CDE': [7, 8, 9],
 'ABCD': [5, 9],
 'ABCE': [9],
 'ABDE': [6, 9],
 'ACDE': [7, 9],
 'BCDE': [8, 9],
 'ABCDE': [9]}

**2.3**. *Самостоятельно* реализуйте алгоритм **Charm**. Используя полученные пары **P** и алгоритм **Charm**, вычислите список всех закрытых наборов **C** (*closed itemsets*). Продемонстрируйте работу алгоритма. Отсортируйте **C** по ключу ```(len(Ci), Ci)``` в порядке возрастания.<br /><br />

In [50]:
def Charm(index_set):
    letters = []
    indices = []
    ale = ''
    for i in index_set:
        if len(i) == 1:
            letters.append(i)
            indices.append(index_set[i])
        
    df = pd.DataFrame(columns=['items'])
    mask = [False] * len(letters)

    for j in range(10):        
        for i, value_i in enumerate(letters):
            if j in index_set[value_i]:
                ale+=value_i
        df.loc[len(df)] = ale
        ale = ''
    return df
Charm(index_set)

,items
0,AC
1,BD
2,CE
3,ACD
4,BCD
5,ABCD
6,ABDE
7,ACDE
8,BCDE
9,ABCDE


**2.4**. В исходных данных добавьте +1 к частоте набора ***CE*** (и ко всем его поднаборам) и повторно получите изначальный набор данных  **T**, множество  **P** и список всех закрытых наборов. После изменения частот также выведите обновленный список наборов и их частот. Значение минимального уровня поддержки оставьте без изменений. Повторная демонстрация работы алгоритма **Charm** не требуется.<br /><br />


In [51]:
data = ['ABCDE', 'ABCD', 'ABDE', 'ACDE', 'BCDE', 'ACD', 'BCD', 'AC', 'BD', 'CE', 'C', 'E']
data.sort(key=lambda x:len(x))
T = pd.DataFrame({'items': data})
print(T)
display(get_index(T))
Charm(get_index(T))

    items
0       C
1       E
2      AC
3      BD
4      CE
5     ACD
6     BCD
7    ABCD
8    ABDE
9    ACDE
10   BCDE
11  ABCDE


{'A': [2, 5, 7, 8, 9, 11],
 'B': [3, 6, 7, 8, 10, 11],
 'C': [0, 2, 4, 5, 6, 7, 9, 10, 11],
 'D': [3, 5, 6, 7, 8, 9, 10, 11],
 'E': [1, 4, 8, 9, 10, 11],
 'AB': [7, 8, 11],
 'AC': [2, 5, 7, 9, 11],
 'AD': [5, 7, 8, 9, 11],
 'AE': [8, 9, 11],
 'BC': [6, 7, 10, 11],
 'BD': [3, 6, 7, 8, 10, 11],
 'BE': [8, 10, 11],
 'CD': [5, 6, 7, 9, 10, 11],
 'CE': [4, 9, 10, 11],
 'DE': [8, 9, 10, 11],
 'ABC': [7, 11],
 'ABD': [7, 8, 11],
 'ABE': [8, 11],
 'ACD': [5, 7, 9, 11],
 'ACE': [9, 11],
 'ADE': [8, 9, 11],
 'BCD': [6, 7, 10, 11],
 'BCE': [10, 11],
 'BDE': [8, 10, 11],
 'CDE': [9, 10, 11],
 'ABCD': [7, 11],
 'ABCE': [11],
 'ABDE': [8, 11],
 'ACDE': [9, 11],
 'BCDE': [10, 11],
 'ABCDE': [11]}

,items
0,C
1,E
2,AC
3,BD
4,CE
5,ACD
6,BCD
7,ABCD
8,ABDE
9,ACDE


### Задание №3

Даны последовательности:

| id              | sequence |
| --------------- | -------- |
| *s<sub>1</sub>* | *ACAAATGAAGTCG*     |
| *s<sub>2</sub>* | *CCTACTGGGT*     |
| *s<sub>3</sub>* | *TTCGTGCTATA*     |
| *s<sub>4</sub>* | *AATGGCAGCCTGGT*     |
| *s<sub>5</sub>* | *AAGCGTTACTTTA*     |

Минимальный уровень поддержки = **4**<br /><br />

In [52]:
data = [
    "ACAAATGAAGTCG",
    "CCTACTGGGT",
    "TTCGTGCTATA",
    "AATGGCAGCCTGGT",
    "AAGCGTTACTTTA"
]

min_support = 4

**3.1**.*Самостоятельно* реализуйте алгоритм **GSP** и найдите все подпоследовательности. Продемонстрируйте, как алгоритм перебирает предложенный набор данных. Оформите этот пункт в виде функции

In [47]:
def GSP(data, min_support):
    frequent_sequences = {}

    for sequence_length in range(1, max(len(s) for s in data) + 1):
        possible_sequences = list(itertools.combinations(set([item for seq in data for item in seq]), sequence_length))
        for seq in possible_sequences:
            support_count = sum(1 for s in data if set(seq).issubset(s))
            if support_count >= min_support:
                frequent_sequences[seq] = support_count
    for key, value in frequent_sequences.items():
        print(key, " : ", value)
GSP(data, min_support)

('T',)  :  5
('A',)  :  5
('C',)  :  5
('G',)  :  5
('T', 'A')  :  5
('T', 'C')  :  5
('T', 'G')  :  5
('A', 'C')  :  5
('A', 'G')  :  5
('C', 'G')  :  5
('T', 'A', 'C')  :  5
('T', 'A', 'G')  :  5
('T', 'C', 'G')  :  5
('A', 'C', 'G')  :  5
('T', 'A', 'C', 'G')  :  5


**3.2**. К исходным данным добавьте последовательность ***TCGTTGT*** и повторно найдите все подпоследовательности. Значение минимального уровня поддержки оставьте без изменений.

In [48]:
data.append('TCGTTGT')
GSP(data, min_support)

('T',)  :  6
('A',)  :  5
('C',)  :  6
('G',)  :  6
('T', 'A')  :  5
('T', 'C')  :  6
('T', 'G')  :  6
('A', 'C')  :  5
('A', 'G')  :  5
('C', 'G')  :  6
('T', 'A', 'C')  :  5
('T', 'A', 'G')  :  5
('T', 'C', 'G')  :  6
('A', 'C', 'G')  :  5
('T', 'A', 'C', 'G')  :  5
